In [14]:
import transformers as T
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from torchmetrics import SpearmanCorrCoef, Accuracy, F1Score
device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f'device = {device}')

device = cuda:0


In [15]:
# fix the random seeds
# The code is generated by ChatGPT
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [16]:
# 有些中文的標點符號在tokenizer編碼以後會變成[UNK]，所以將其換成英文標點
token_replacement = [
    ["：" , ":"],
    ["，" , ","],
    ["“" , "\""],
    ["”" , "\""],
    ["？" , "?"],
    ["……" , "..."],
    ["！" , "!"]
]

In [17]:
class SemevalDataset(Dataset):
    def __init__(self, split="train") -> None:
        super().__init__()
        assert split in ["train", "validation", "test"]
        self.data = load_dataset(
            "sem_eval_2014_task_1", split=split, cache_dir="./cache/", trust_remote_code=True
        ).to_list()

    def __getitem__(self, index):
        d = self.data[index]
        # 把中文標點替換掉
        for k in ["premise", "hypothesis"]:
            for tok in token_replacement:
                d[k] = d[k].replace(tok[0], tok[1])
        return d

    def __len__(self):
        return len(self.data)

data_sample = SemevalDataset(split="train").data[:3]
print(f"Dataset example: \n{data_sample[0]} \n{data_sample[1]} \n{data_sample[2]}")

Dataset example: 
{'sentence_pair_id': 1, 'premise': 'A group of kids is playing in a yard and an old man is standing in the background', 'hypothesis': 'A group of boys in a yard is playing and a man is standing in the background', 'relatedness_score': 4.5, 'entailment_judgment': 0} 
{'sentence_pair_id': 2, 'premise': 'A group of children is playing in the house and there is no man standing in the background', 'hypothesis': 'A group of kids is playing in a yard and an old man is standing in the background', 'relatedness_score': 3.200000047683716, 'entailment_judgment': 0} 
{'sentence_pair_id': 3, 'premise': 'The young boys are playing outdoors and the man is smiling nearby', 'hypothesis': 'The kids are playing outdoors near a man with a smile', 'relatedness_score': 4.699999809265137, 'entailment_judgment': 1}


In [18]:
# Define the hyperparameters
lr = 3e-5
epochs = 10
train_batch_size = 50
validation_batch_size = 50
test_batch_size = 50

In [19]:
tokenizer = T.BertTokenizer.from_pretrained("bert-base-uncased")

In [20]:
# TODO1: Create batched data for DataLoader
# `collate_fn` is a function that defines how the data batch should be packed.
# This function will be called in the DataLoader to pack the data batch.

def collate_fn(batch):
    # TODO1-1: Implement the collate_fn function
    # Write your code here
    # The input parameter is a data batch (tuple), and this function packs it into tensors.
    # Use tokenizer to pack tokenize and pack the data and its corresponding labels.
    # Return the data batch and labels for each sub-task.

    # Define the storing structure of sentence_pairs from ChatGPT -> giving me the idea of [SEP]
    sentence_pairs = [item['premise'] + " [SEP] " + item['hypothesis'] for item in batch]
    relatedness_scores = [item['relatedness_score'] for item in batch]
    entailment_judgments = [item['entailment_judgment'] for item in batch]
    
    
    encoded_inputs = tokenizer(
        sentence_pairs,
        return_tensors='pt', # return the result with pytorch tensor
        padding=True,
        truncation=True,
    )
    
    
    relatedness_scores_tensor = torch.tensor(relatedness_scores, dtype=torch.float)
    entailment_judgments_tensor = torch.tensor(entailment_judgments, dtype=torch.long)
    
    return {
        'input_ids': encoded_inputs['input_ids'],
        'token_type_ids': encoded_inputs['token_type_ids'],
        'attention_mask': encoded_inputs['attention_mask'],
        'relatedness_score': relatedness_scores_tensor,
        'entailment_judgment': entailment_judgments_tensor
    }

    
# TODO1-2: Define your DataLoader
ds_train = SemevalDataset(split="train").data
ds_validation = SemevalDataset(split="validation").data
dl_train = DataLoader(ds_train, batch_size=train_batch_size, collate_fn=collate_fn) # Write your code here
dl_validation = DataLoader(ds_validation, batch_size=validation_batch_size, collate_fn=collate_fn) # Write your code here

In [21]:
# TODO2: Construct your model
class MultiLabelModel(torch.nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Write your code here
        # Define what modules you will use in the model
        self.bert = T.BertModel.from_pretrained("bert-base-uncased")
        self.linear_list = torch.nn.Sequential(
                                               torch.nn.Linear(self.bert.config.hidden_size, 2000), 
                                               torch.nn.Linear(2000, 500), 
                                               torch.nn.Linear(500, 125)
                                               )
        self.linear_relatedness = torch.nn.Linear(125, 1)
        self.linear_entailment = torch.nn.Linear(125, 3)
    def forward(self, **kwargs):
        # Write your code here
        # Forward pass
        # Forward pass through BERT
        input_ids = kwargs['input_ids']
        attention_mask = kwargs['attention_mask']
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs.pooler_output
        
        x = self.linear_list(x)
        
        # Predict relatedness score
        relatedness_score = self.linear_relatedness(x).squeeze(-1)
        
        # Predict entailment judgment
        entailment_judgment = self.linear_entailment(x)
        
        return {
            'relatedness_score': relatedness_score,
            'entailment_judgment': entailment_judgment
        }

In [22]:
model = MultiLabelModel().to(device)

In [23]:
# TODO3: Define your optimizer and loss function

# TODO3-1: Define your Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)# Write your code here

# TODO3-2: Define your loss functions (you should have two)
# Write your code here
MSE = torch.nn.MSELoss()
CrossEntorpy = torch.nn.CrossEntropyLoss()

# scoring functions
spc = SpearmanCorrCoef().to(device)
acc = Accuracy(task="multiclass", num_classes=3).to(device)
f1 = F1Score(task="multiclass", num_classes=3, average='macro').to(device)

c:\hw_and_code\NaturalLanguageProcess\.venv\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [24]:
for ep in range(epochs):
    pbar = tqdm(dl_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    model.train()
    # TODO4: Write the training loop
    # Write your code here
    # train your model
    # clear gradient
    # forward pass
    # compute loss
    # back-propagation
    # model optimization
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        relatedness_scores = batch['relatedness_score'].to(device)
        entailment_judgments = batch['entailment_judgment'].to(device)
        
        optimizer.zero_grad()
        pred = model(input_ids = input_ids, attention_mask = attention_mask)

        loss_regression = MSE(pred['relatedness_score'], relatedness_scores)
        loss_classification = CrossEntorpy(pred['entailment_judgment'], entailment_judgments)
        # weight be 1 means only trains the resgression part, and 0 means only trains the classification part
        # 0.5 means training at the same time -> multi-output
        # get the idea of accumulate the result of both loss functions by ChatGPT
        loss = 0.5 * loss_regression + 0.5 * loss_classification
        loss.backward()
        optimizer.step()
        

    pbar = tqdm(dl_validation)
    pbar.set_description(f"Validation epoch [{ep+1}/{epochs}]")
    model.eval()
    # TODO5: Write the evaluation loop
    # Write your code here
    # Evaluate your model
    # Output all the evaluation scores (SpearmanCorrCoef, Accuracy, F1Score)
    with torch.no_grad():
        for batch in pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            relatedness_score = batch['relatedness_score'].to(device)
            entailment_judgment = batch['entailment_judgment'].to(device)
            
            # Forward pass
            pred = model(input_ids=input_ids, attention_mask=attention_mask)
            
            # Scoring
            spc.update(relatedness_score, pred['relatedness_score'])
            acc.update(entailment_judgment, torch.argmax(pred["entailment_judgment"], dim=-1))
            f1.update(entailment_judgment, torch.argmax(pred["entailment_judgment"], dim=-1))
        
        print(f'spc = {spc.compute()}, acc = {acc.compute()}, f1 = {f1.compute()}')
        spc.reset()
        acc.reset()
        f1.reset()
    # torch.save(model, f'./ep{ep}.ckpt')

Validation epoch [1/10]: 100%|██████████| 10/10 [00:00<00:00, 30.67it/s]


spc = 0.7457163333892822, acc = 0.5640000104904175, f1 = 0.24040921032428741


Validation epoch [2/10]: 100%|██████████| 10/10 [00:00<00:00, 30.46it/s]


spc = 0.7887118458747864, acc = 0.699999988079071, f1 = 0.4895747900009155


Validation epoch [3/10]: 100%|██████████| 10/10 [00:00<00:00, 30.56it/s]


...

spc = 0.8011781573295593, acc = 0.8299999833106995, f1 = 0.8290132284164429


Validation epoch [9/10]: 100%|██████████| 10/10 [00:00<00:00, 30.40it/s]


spc = 0.8117408156394958, acc = 0.8500000238418579, f1 = 0.8448750972747803


Validation epoch [10/10]: 100%|██████████| 10/10 [00:00<00:00, 30.54it/s]

spc = 0.8214926719665527, acc = 0.8600000143051147, f1 = 0.8509469628334045


For test set predictions, you can write perform evaluation simlar to #TODO5.

In [25]:
ds_test = SemevalDataset(split="test").data
dl_test = DataLoader(dataset=ds_test, batch_size=test_batch_size, collate_fn=collate_fn)

pbar = tqdm(dl_test)
pbar.set_description(f"Test")
model.eval()

error_data = {}
correct_data = {}

with torch.no_grad():
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        relatedness_score = batch['relatedness_score'].to(device)
        entailment_judgment = batch['entailment_judgment'].to(device)
        
        output = model(input_ids=input_ids, attention_mask=attention_mask)
        
       # calculate scores
        spc.update(relatedness_score, output['relatedness_score'])
        acc.update(entailment_judgment, torch.argmax(output["entailment_judgment"], dim=-1))
        f1.update(entailment_judgment, torch.argmax(output["entailment_judgment"], dim=-1))

        mask1 = relatedness_score != output['relatedness_score']
        mask2 = entailment_judgment != torch.argmax(output["entailment_judgment"], dim=-1)
        mask = mask1 & mask2
        if not error_data:
            error_data['relatedness_score'] = output['relatedness_score'][mask]
            error_data['entailment_judgment'] = torch.argmax(output["entailment_judgment"], dim=-1)[mask]
            correct_data['relatedness_score'] = relatedness_score[mask]
            correct_data['entailment_judgment'] = entailment_judgment[mask]

    print(f'\nspc = {spc.compute()}')
    print(f'acc = {acc.compute()}')
    print(f'f1 = {f1.compute()}')
    spc.reset()
    acc.reset()
    f1.reset()

Test: 100%|██████████| 99/99 [00:02<00:00, 37.38it/s]


spc = 0.8263977766036987
acc = 0.8751776218414307
f1 = 0.8656327128410339


In [26]:
# print out errors in the first batch

ER = error_data['relatedness_score']
EE = error_data['entailment_judgment']
CR = correct_data['relatedness_score']
CE = correct_data['entailment_judgment']
errList = list(map(lambda x: (float(x[0]), int(x[1])), list(zip(ER, EE))))
ansList = list(map(lambda x: (float(x[0]), int(x[1])), list(zip(CR, CE))))

# print(errList[0])
# print(ansList[0])
print("error data = ", end="")
for i in range(len(errList)):
    if abs(errList[i][0] - ansList[i][0]) <= 0.5:
        print(f'({errList[i][0]:.4f}, {errList[i][1]})', end=" ")
print()
print("truth data = ", end="")
for i in range(len(errList)):
    if abs(errList[i][0] - ansList[i][0]) <= 0.5:
        print(f'({ansList[i][0]:.4f}, {ansList[i][1]})', end=" ")
print()

error data = (4.0955, 1) (3.5848, 0) (3.4813, 0) (4.4089, 1) (3.9086, 0) (3.4206, 0) 
truth data = (3.6650, 0) (3.9000, 2) (3.5000, 2) (4.0000, 0) (4.4000, 1) (3.6000, 2) 
